In [113]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.chrome.options import Options
import uuid
import os
from datetime import datetime
import re
import pandas as pd
from platform import release
from sqlalchemy import create_engine
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.orm import relationship


#driver = webdriver.Chrome()
URL = "https://www.metacritic.com/movie/the-matrix"
#URL = "https://www.metacritic.com/movie/happening"

chrome_options = Options()
chrome_options.add_argument("--headless")
driver = webdriver.Chrome(options=chrome_options)

driver.get(URL)

delay = 10

print (driver)

time.sleep(1)

try: # if the reject cookies button is there then click it
    reject_cookies_btn = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="onetrust-reject-all-handler"]')))
    reject_cookies_btn.click()

except NoSuchElementException: # open the cookies settings, then click reject
    cookies_settings_btn = driver.find_element(by=By.XPATH, value='//*[@id="ot-sdk-btn"]')
    cookies_settings_btn.click()
    reject_cookies_btn = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//*[@id="onetrust-pc-sdk"]/div[3]/div[1]/button[1]')))
    reject_cookies_btn.click()

except TimeoutException:
    print ("Loading cookies took too much time!")

except:
    pass

def save_raw_data( data): #TODO use try, except block

    if os.path.isdir('raw_data'):
        if os.path.exists(os.path.join(os.getcwd(), 'raw_data', data['friend_id'])):
            with open(data['friend_id']+ ".json", "w") as file:
                file.write(data)

    else:
        create_data_folder( data)

def create_data_folder( data):

    if not os.path.isdir('raw_data'):
        os.makedirs('raw_data')
        save_raw_data(data)


def get_image_src(driver):
    img_src = driver.find_element(By.XPATH, value = '//img[@class="summary_img"]').get_attribute("src")
    return img_src

# movies = driver.find_elements(By.XPATH, '//div[@class="title_bump"]//a[@class="title"]')
# links = [movie.get_attribute('href') for movie in movies]
# src = get_image_src(driver)
# print(src)

# data = {'uuid':[],'friend_id':[],'title':[],'metascore':[],'release_date':[],'starring':[],
#                 'director':[],'genres':[],'rating':[],'runtime':[],'images':[]}

# data['uuid'] = str(uuid.uuid4())

# last_part_url = driver.current_url.split("/")[-1]
# data['friend_id'] = last_part_url

# data['title'] = driver.find_element(By.XPATH, value = '//div[@class="product_page_title oswald"]/h1').text
# print(data['title'])

# #data['metascore'] = driver.find_element(By.XPATH, value = '//span[contains(@class, "metascore_w larger"]').text
# data['metascore'] = driver.find_element(By.XPATH, value = '//a[@class="metascore_anchor"]/span').text
# print(data['metascore'])

# data['release_date'] = driver.find_element(By.XPATH, value = '//span[@class="release_date"]/span[2]').text
# print(data['release_date'])

# actors = driver.find_elements(By.XPATH, value = '//div[@class="summary_cast details_section"]/span[2]/a')
# data['starring'] = [actor.text for actor in actors]
# #for actor in actors: data['starring'].append(actor.text) 
# print (data['starring'])

# directors = driver.find_elements(By.XPATH, value = '//div[@class="director"]/a')
# # data['director'] = driver.find_element(By.XPATH, value = '//div[@class="director"]/a').text
# data['director'] = [director.text for director in directors]
# print (data['director'])

# genres = driver.find_elements(By.XPATH, value = '//div[@class="genres"]/span[2]/span')
# data['genres'] = [genre.text for genre in genres]
# print (data['genres'])

# rating = driver.find_element(By.XPATH, value = '//div[@class="rating"]/span[2]').text
# data['rating'] = rating
# print (rating)

# runtime = driver.find_element(By.XPATH, value = '//div[@class="runtime"]/span[2]').text
# data['runtime']  = runtime
# print (runtime)

# print (data)

# print(type(get_image_src(driver)))

def return_element_if_exists(driver, xpath):
    try:
        ele = driver.find_element(By.XPATH, value = xpath)
    except NoSuchElementException:
        return None
    return ele

def get_film_title(driver) -> str:
        """Finds and returns the current film's title as a string."""
        xpath = '//div[@class="product_page_title oswald"]/h1'
        result = return_element_if_exists(driver, xpath)
        
        ret = result.text if result else result
        return ret


def return_elements_if_exist(driver, xpath):
    try:
        eles = driver.find_elements(By.XPATH, value = xpath)
        ele_list = [ele.text for ele in eles]
    except NoSuchElementException:
        return None
    return ele_list

def get_actors(driver):
        """Finds and returns the current film's starring actors as a List."""
        xpath = '//div[@class="summary_cast details_section"]/span[2]/a'
        return return_elements_if_exist(driver, xpath)

def get_summary_img(driver) -> str:
        """Finds and returns the current film's summary image source as a string."""
        xpath = '//img[@class="summary_img"]'
        result = return_element_if_exists(driver, xpath)
        ret = result.text if result else result
        return ret

def get_metascore(driver) -> str:
        """Finds and returns the current film's metascore as a string."""
        xpath = '//a[@class="metascore_anchor"]/span'
        result = return_element_if_exists(driver, xpath)
        ret = int(result.text) if result else result
        return ret

def get_release_date(driver) -> str:
        """Finds and returns the current film's release date as a string."""
        xpath = '//span[@class="release_date"]/span[2]'
        result = return_element_if_exists(driver, xpath)
        release_date = result.text if result else result
        release_date = datetime.strptime(release_date, '%B %d, %Y').date()
        return release_date

def get_runtime(driver) -> str:
        """Finds and returns the current film's runtime as a string."""
        xpath = '//div[@class="runtime"]/span[2]'
        result = return_element_if_exists(driver, xpath)
        runtime = result.text if result else result
        runtime = int(re.sub("[^0-9]", "", runtime))
        return runtime

def get_directors(self):
        """Finds and returns the current film's directors as a List."""
        xpath = '//div[@class="director"]/a'
        return return_elements_if_exist(driver, xpath)

def get_genres(self):
        """Finds and returns the current film's genres as a List"""
        xpath = '//div[@class="genres"]/span[2]/span'
        return return_elements_if_exist(driver, xpath)

        
film_title = get_film_title(driver)
print(film_title)
actors = get_actors(driver)
print(actors)

print(get_summary_img(driver))

genres = get_genres(driver)
directors = get_directors(driver)
release_date = get_release_date(driver)
metascore = get_metascore(driver)
runtime = get_runtime(driver)

print(directors)
print(release_date)
print(type(release_date))

print(metascore)
print(type(metascore))

print(runtime)
print(type(runtime))


uuid = 'ab76ecc3-3467-4de1-b1a4-c0545be1da26'


def does_row_exist(engine, unique_key, value, table):
    print("HELLO")
    result = engine.execute(f"""SELECT 1
                                FROM {table}
                                WHERE {unique_key} = '{value}';""").fetchall()
    if result == []: 
        return False
    else:
        return True
    
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'aicore-datapipe-db.cu3ccladgvto.eu-west-2.rds.amazonaws.com' # Change it for your AWS endpoint
USER = 'postgres'
PASSWORD = 'shroomchickenoctopusorange'
PORT = 5432
DATABASE = 'postgres'
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")
engine.connect()

if not does_row_exist(engine, 'uuid', uuid, 'film'):
    engine.execute(f'''INSERT INTO film (uuid, friendly_id, title, metascore, release_date, rating, runtime)
                        VALUES ('{uuid}', 'a-chiara', 'A Chiara', '{metascore}', '{release_date}', 'R', '{runtime}');''')

for actor in actors:
    if not does_row_exist(engine, 'actor_name', actor, 'actor'):
        engine.execute(f'''INSERT INTO actor (actor_name)
                            VALUES ('{actor}');''')

    actor_id = engine.execute(f"""SELECT id FROM actor WHERE actor_name = '{actor}';""").first()[0]
    if not does_row_exist(engine, 'film_id', uuid, 'actor_link') and not does_row_exist(engine, 'actor_id', actor_id, 'actor_link'):
        engine.execute(f'''INSERT INTO actor_link (film_id, actor_id)
                                VALUES ('{uuid}', '{actor_id}');''')

for director in directors:
    if not does_row_exist(engine, 'director_name', director, 'director'):
        engine.execute(f'''INSERT INTO director (director_name)
                            VALUES ('{director}');''')

    director_id = engine.execute(f"""SELECT id FROM director WHERE actor_name = '{director}';""").first()[0]
    if not does_row_exist(engine, 'film_id', uuid, 'director_link') and not does_row_exist(engine, 'director_id', director_id, 'director_link'):
        engine.execute(f'''INSERT INTO director_link (film_id, director_id)
                                VALUES ('{uuid}', '{director_id}');''')

for actor in actors:
    if not does_row_exist(engine, 'actor_name', actor, 'actor'):
        engine.execute(f'''INSERT INTO actor (actor_name)
                            VALUES ('{actor}');''')

    actor_id = engine.execute(f"""SELECT id FROM actor WHERE actor_name = '{actor}';""").first()[0]
    if not does_row_exist(engine, 'film_id', uuid, 'actor_link') and not does_row_exist(engine, 'actor_id', actor_id, 'actor_link'):
        engine.execute(f'''INSERT INTO actor_link (film_id, actor_id)
                                VALUES ('{uuid}', '{actor_id}');''')





<selenium.webdriver.chrome.webdriver.WebDriver (session="6a3796c5e4725c6caf01d3e6b1e96e50")>
The Matrix
['Carrie-Anne Moss', 'Keanu Reeves', 'Laurence Fishburne']

['Lana Wachowski', 'Lilly Wachowski']
1999-03-31
<class 'datetime.date'>
73
<class 'int'>
136
<class 'int'>
HELLO
[(1,)]
result is not []
HELLO
[(1,)]
result is not []
HELLO
[(1,), (1,), (1,)]
result is not []
HELLO
[(1,)]
result is not []
HELLO
[(1,), (1,), (1,)]
result is not []
HELLO
[(1,)]
result is not []
HELLO
[(1,), (1,), (1,)]
result is not []


In [57]:
import pandas as pd
from platform import release
from sqlalchemy import create_engine
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.orm import relationship

the_uuid = 'ab76ecc3-3467-4de1-b1a4-c0545be1da26'
the_name = 'Laurence Fishburne'

def check(engine, thing, table):
    engine.connect()
    print(engine.execute(f"""SELECT * FROM {table} WHERE uuid='{the_uuid}';""").fetchall())

def does_row_exist(engine, unique_key, value, table):
    result = engine.execute(f"""SELECT 1
                                FROM {table}
                                WHERE {unique_key} = '{value}';""").fetchall()
    if result == []: return True
    else: return False
    
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'aicore-datapipe-db.cu3ccladgvto.eu-west-2.rds.amazonaws.com' # Change it for your AWS endpoint
USER = 'postgres'
PASSWORD = 'shroomchickenoctopusorange'
PORT = 5432
DATABASE = 'postgres'
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")
engine.connect()

if not does_row_exist(engine, 'uuid', the_uuid, 'film'):
    engine.execute(f'''INSERT INTO film (uuid, friendly_id, title, metascore, release_date, rating, runtime)
                        VALUES ('{uuid}', 'a-chiara', 'A Chiara', '{metascore}', '{release_date}', 'R', '{runtime}');''')

for genre in genres:
    if not does_row_exist(engine, 'genre_name', the_name, 'genre'):
        engine.execute(f'''INSERT INTO genre (genre_name)
                            VALUES ('{the_name}');''')

    genre_id = engine.execute(f"""SELECT id FROM genre WHERE genre_name = '{the_name}';""").first()[0]
    engine.execute(f'''INSERT INTO genre_link (film_id, genre_id)
                            VALUES ('{uuid}', '{genre_id}');''')

pdfilms = pd.read_sql_table('film', engine)
pdgenres = pd.read_sql_table('genre', engine)
pdgenrelink = pd.read_sql_table('genre_link', engine)
#pdfilms.head(10)
# pdgenres.head(10)
# pdgenrelink.head(10)

[('ab76ecc3-3467-4de1-b1a4-c0545be1da26', 'a-chiara', 'A Chiara', 73, datetime.date(1999, 3, 31), 'R', 136)]


In [2]:
#
# CELL TO PERFORM MISC DB OPERATIONS ##########################################################################
#


import pandas as pd
from platform import release
from sqlalchemy import create_engine
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.orm import relationship

the_uuid = 'ab76ecc3-3467-4de1-b1a4-c0545be1da26'
the_name = 'Laurence Fishburne'

def check(engine, thing, table):
    engine.connect()
    print(engine.execute(f"""SELECT * FROM {table} WHERE uuid='{the_uuid}';""").fetchall())

def does_row_exist(engine, unique_key, value, table):
    result = engine.execute(f"""SELECT 1
                                FROM {table}
                                WHERE {unique_key} = '{value}';""").fetchall()
    if result == []: return True
    else: return False
    
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'aicore-datapipe-db.cu3ccladgvto.eu-west-2.rds.amazonaws.com' # Change it for your AWS endpoint
USER = 'postgres'
PASSWORD = 'shroomchickenoctopusorange'
PORT = 5432
DATABASE = 'postgres'
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")
engine.connect()

# engine.execute("""DELETE FROM genre_link;""")
# engine.execute("""DELETE FROM director_link;""")
# engine.execute("""DELETE FROM actor_link;""")
# engine.execute("""DELETE FROM genre;""")
# engine.execute("""DELETE FROM actor;""")

# engine.execute("""DELETE FROM director;""")

# engine.execute("""DELETE FROM film;""")

pdfilms = pd.read_sql_table('film', engine)
pdactors = pd.read_sql_table('actor', engine)
pdactorlink = pd.read_sql_table('actor_link', engine)
pddirectors = pd.read_sql_table('director', engine)
pddirectorlink = pd.read_sql_table('director_link', engine)
pdgenrelink = pd.read_sql_table('genre_link', engine)
pdgenres = pd.read_sql_table('genre', engine)
pdfilms.head(60)
#pdactors.head(50)
# pddirectors.head(60)
# pddirectorlink.head(60)
pdactorlink.head(30)
#pdgenrelink.head(60)
#pdgenres.head(60)




OperationalError: (psycopg2.OperationalError) connection to server at "aicore-datapipe-db.cu3ccladgvto.eu-west-2.rds.amazonaws.com" (3.10.184.74), port 5432 failed: Connection timed out
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [ ]:
import pathlib
import os
import json
import requests

nice_data = {'uuid': '992ae5b5-d115-4c48-984d-b73757f17f2b', 'friend_id': 'the-matrix', 'title': 'The Matrix', 'metascore': '73', 'release_date': 'March 31, 1999', 'starring': ['Carrie-Anne Moss', 'Keanu Reeves', 'Laurence Fishburne'], 'director': ['Lana Wachowski', 'Lilly Wachowski'], 'genres': ['Action', 'Adventure', 'Sci-Fi', 'Thriller'], 'rating': 'R', 'runtime': '136 min', 'images': ['https://static.metacritic.com/images/products/movies/5/14d38f138eb320954cd1e07d0449e5a6-250h.jpg']}

def save_raw_data( data): #TODO use try, except block

    if os.path.isdir('raw_data'):
        filename = "raw_data/" + data['friend_id'] + ".json"
        print(filename)
        with open(filename, "w") as file:
            file.write(json.dumps(data))

    else:
        create_data_folder( data)

def create_data_folder( data):

    if not os.path.isdir('raw_data'):
        os.makedirs('raw_data')
        save_raw_data(data)

def save_image(data):

    for x, image in enumerate(data['images']):
        filename = "raw_data/images/{}_{}.jpg".format(data['friend_id'], str(x))
        img_data = requests.get(image).content
        with open(filename, 'wb') as handler:
            handler.write(img_data)



save_image(nice_data)


In [9]:
blow = {}

blow['frosh'] = "gost"
blow['low'] = None

print(blow)


{'frosh': 'gost', 'low': None}


In [30]:
mylist = []
mylist.append("")
print(mylist)

for thing in mylist:
    if thing != "":
        print("yes")
    else:
        print("no")

['']
no


In [34]:
filename = 'hello.json'

hi =(''.join(['raw_data', filename]))
print(hi)

raw_datahello.json


In [41]:
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
HOST = 'localhost'
USER = 'postgres'
PASSWORD = 'drillroofteasteel'
DATABASE = 'Pagila'
PORT = 5432
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")
engine.connect()
# inspector = inspect(engine)
# inspector.get_table_names()
# engine.execute('''SELECT * FROM genre''').fetchall()

genres = pd.read_sql_table('genre', engine)
genres.head(10)

,actor_id,first_name,last_name,last_update
0,1,PENELOPE,GUINESS,2006-02-15 09:34:33
1,2,NICK,WAHLBERG,2006-02-15 09:34:33
2,3,ED,CHASE,2006-02-15 09:34:33
3,4,JENNIFER,DAVIS,2006-02-15 09:34:33
4,5,JOHNNY,LOLLOBRIGIDA,2006-02-15 09:34:33
5,6,BETTE,NICHOLSON,2006-02-15 09:34:33
6,7,GRACE,MOSTEL,2006-02-15 09:34:33
7,8,MATTHEW,JOHANSSON,2006-02-15 09:34:33
8,9,JOE,SWANK,2006-02-15 09:34:33
9,10,CHRISTIAN,GABLE,2006-02-15 09:34:33


In [2]:
from platform import release
from sqlalchemy import create_engine
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String
from sqlalchemy import *
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()
from sqlalchemy.orm import relationship
DATABASE_TYPE = 'postgresql'
DBAPI = 'psycopg2'
ENDPOINT = 'aicore-datapipe-db.cu3ccladgvto.eu-west-2.rds.amazonaws.com' # Change it for your AWS endpoint
USER = 'postgres'
PASSWORD = 'shroomchickenoctopusorange'
PORT = 5432
DATABASE = 'postgres'
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{ENDPOINT}:{PORT}/{DATABASE}")
engine.connect()

class Film(Base):
    __tablename__ = 'film'
    uuid = Column(VARCHAR, primary_key = True)
    friendly_id = Column(VARCHAR)
    title = Column(VARCHAR)
    metascore = Column(Integer)
    release_date = Column(Date)
    rating = Column(VARCHAR)
    runtime = Column(Integer)
    genres = relationship('Genre', secondary = 'genre_link')
    directors = relationship('Director', secondary = 'director_link')
    actors = relationship('Actor', secondary = 'actor_link')

class Genre(Base):
    __tablename__ = 'genre'
    id = Column(Integer, primary_key = True)
    genre_name = Column(String)
    films = relationship('Film', secondary = 'genre_link')

class GenreLink(Base):
    __tablename__ = 'genre_link'
    film_id = Column(VARCHAR, ForeignKey('film.uuid'), primary_key = True)
    genre_id = Column(Integer, ForeignKey('genre.id'), primary_key = True)

class Director(Base):
    __tablename__ = 'director'
    id = Column(Integer, primary_key = True)
    director_name = Column(String)
    films = relationship('Film', secondary = 'director_link')

class DirectorLink(Base):
    __tablename__ = 'director_link'
    film_id = Column(VARCHAR, ForeignKey('film.uuid'), primary_key = True)
    director_id = Column(Integer, ForeignKey('director.id'), primary_key = True)

class Actor(Base):
    __tablename__ = 'actor'
    id = Column(Integer, primary_key = True)
    actor_name = Column(String)
    films = relationship('Film', secondary = 'actor_link')

class ActorLink(Base):
    __tablename__ = 'actor_link'
    film_id = Column(VARCHAR, ForeignKey('film.uuid'), primary_key = True)
    actor_id = Column(Integer, ForeignKey('actor.id'), primary_key = True)


Base.metadata.create_all(engine)



In [15]:
from datetime import datetime
import re
import dateutil

release_date = "March 31, 1999"

metascore = "73"

runtime = "136 min"


date_object = datetime.strptime(release_date, '%B %d, %Y').date()
print(type(date_object))
print(date_object)  # printed in default formatting

runtime = int(re.sub("[^0-9]", "", runtime))

print(runtime)
print(type(runtime))

<class 'datetime.date'>
1999-03-31
136
<class 'int'>


In [11]:
actors = ['John Wayne', 'Tom Keaton', 'Christian Bale']
directors = ['Steven Speel', 'Lana Wachowski']
genres = ['Action', 'Adventure']

actors = 'John Wayne'
directors = 'Steven Speel'
genres = 'Action'

byes = b'byesbyesbyes'

myDict = {'actors': actors, 'directors': directors, 'genres': genres, 'bytes' : byes}

print(myDict)

for shit, fuck in myDict.items():
    print(shit)
    print (fuck)
    # for point in value:
    #     print (point)

    


{'actors': 'John Wayne', 'directors': 'Steven Speel', 'genres': 'Action', 'bytes': b'byesbyesbyes'}
actors
John Wayne
directors
Steven Speel
genres
Action
bytes
b'byesbyesbyes'


In [158]:
import configparser

config = configparser.ConfigParser()
# read the configuration file
config.read('my_config.ini')
# get all the connections
#config.sections()
dbpw = config.get('DB','user')

print(dbpw)

postgres


In [1]:
import socket

a_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

port = 4444
location = ("127.0.0.1", port)
check = a_socket.connect_ex(location)

if check == 0:
   print("Port is open")
else:
   print("Port is not open")

Port is not open


In [10]:
def get_local_upload_choices():
        """Gets the user's choices to save data to machine locally, upload directly to RDS or both.
        """
        local = "y"
        upload = "n"

        return local, upload

choices = get_local_upload_choices()
print(choices)
print(type(choices))

('y', 'n')
<class 'tuple'>


In [17]:
def the_func():

    mydict = {}
    mylist = []

    mydict['hello'] = 'hi'
    mydict['goodbye'] = 'bye'

    mylist.append("yo")
    mylist.append("bro")

    return mydict, mylist

newdict, newlist = the_func()

print(newdict)
print(newlist)

{'hello': 'hi', 'goodbye': 'bye'}
['yo', 'bro']


In [20]:
from datetime import datetime

def get_release_date(result) -> (str | None):
        """Finds and returns the current film's release date as a string."""
        xpath = '//span[@class="release_date"]/span[2]'
        release_date = datetime.strptime(
            result, '%B %d, %Y').date() if result else result
        return release_date

get_release_date('TBA')

AttributeError: 'str' object has no attribute 'text'